# Create weather-dependent smartds files (at peak demand timestep)

This notebook does the following:
1. Define path to opendss ditribution data region folder
2. Load measured and predicted electricity buildings demand data
3. Convert building kw timeseries to smart-ds format kw and kvar loadshapes and max values
4. Extract temp at each timestep (hourly)
5. Create smart-ds LineCodes.dss and Transformers.dss using temp at each timestep
6. Create smart-ds csv profiles, Loads.dss, LoadShapes.dss files using buildings demand predictions
7. Create master.dss file with paths to new load.dss and loadshapes.dss + LineCodes.dss and Transformers.dss

## Initialize files

### Import packages

In [3]:
import time
import datetime
import os
import numpy as np 
import glob
import re
import shutil
import pandas as pd
import joblib
import yaml
import matplotlib.pyplot as plt
import math
import gc
import sys
from collections import defaultdict

from opendssdirect import dss
from src import input_ops
from src import opendss_ops
from src import file_ops
from src import df_ops

### Load config file with scenarios and parameters 

In [1]:
config_file_name = 'opendss_config1'; config_path = f"/home/aviadf/opendss/notebooks/config/{config_file_name}.yaml"; config = input_ops.load_config(config_path)

TGW_years_scenarios = config['TGW_years_scenarios'] 

CITY_REGIONS_TO_RUN = config['CITY_REGIONS_TO_RUN']
    
demand_mode = config['demand_mode'] 
line_rating_mode = config['line_rating_mode'] p
transformers_rating_mode = config['transformers_rating_mode'] 

aggregation_level = config['aggregation_level']

smart_ds_year = config['smart_ds_years'][0]
start_month = config['start_month']
end_month = config['end_month']
smart_ds_load_path = config['smart_ds_load_path'] + f"/{smart_ds_year}" # path to procesed smart-ds resstock data 

input_data_prediction_path = config['input_data_prediction_path']

solution_mode = config['solution_mode'] # yearly | snapshot

## Define variables to create list of mdh to run
start_month_mdh = config['start_month_mdh'] 
end_month_mdh = config['end_month_mdh']
top_percent_mdh = config['top_percent_mdh']

## Load dictionary, sort by total city aggregated buildings demand, extract mdh of top % load hours
## load demand data 
regional_demand_weather_ampacity_path = smart_ds_load_path + f"/all_cities/aggregated_demand"
regional_demand_weather_ampacity_all_cities = joblib.load(os.path.join(regional_demand_weather_ampacity_path, "regional_demand_weather_ampacity_all_cities"))
## Sort your dictionary by aggregated total demand  
regional_demand_weather_ampacity_all_cities_sorted = df_ops.sort_nested_dict_dfs(regional_demand_weather_ampacity_all_cities, "aggregated_predicted_buildings_total_kw", ascending=False)
top_n_hours = int(np.ceil(8760*top_percent_mdh/100)) # calculate top city demand hours to run (top_percent_mdh% of hours of the year)
## Define start and end load hours to run
start_row_percent = config['start_row_percent'] # define percentage of last run to avoid running same mdh again, e.g., start_row_percent = 1 means that loop will start from the 99th percentile hours
start_row_idx = int(np.ceil(8760*start_row_percent/100)) # index from which to start loop of load hours
# start_row_idx = 0 # index from which to start loop of load hours
end_row_idx = top_n_hours 

print(f"TGW_years_scenarios: {TGW_years_scenarios} \nsmart_ds_year:{smart_ds_year} \nsolution_mode:{solution_mode} \ncity region: {CITY_REGIONS_TO_RUN} \n Demand mode: {demand_mode} \nline_rating_mode: {line_rating_mode} \ntransformers_rating_mode: {transformers_rating_mode}")

print(f"\n\ntop_percent_mdh: {top_percent_mdh}%, top_n_hours: {top_n_hours}, start_row_idx:{start_row_idx} ({start_row_percent}%), end_row_idx:{end_row_idx} ({top_percent_mdh}%) \n")

## Create and compare mdh for different climate scenario and years

In [2]:
list_of_mdh = {}
for TGW_weather_year, TGW_scenarios in TGW_years_scenarios.items():
    for TGW_scenario in TGW_scenarios:
        list_of_mdh[(TGW_weather_year, TGW_scenario)] = {}
        print(f"--- Creating list for TGW_scenario:{TGW_scenario} TGW_weather_year:{TGW_weather_year} --- \n")
        for city, regions in CITY_REGIONS_TO_RUN.items():
            ## create list of mdh for top % hours
            df_city = regional_demand_weather_ampacity_all_cities_sorted[(TGW_weather_year, TGW_scenario)][city]   # load regional demand data to later create list of mdh for top % hours  
            list_of_mdh[(TGW_weather_year, TGW_scenario)][(city)] = df_ops.get_top_n_mdh(df_city, top_n_hours, start_month_mdh, end_month_mdh)

In [3]:
# Example: pick two lists of mdh
mdh_list_1 = list_of_mdh[('2058', 'rcp45hotter')]['GSO']
mdh_list_2 = list_of_mdh[('2058', 'rcp85hotter')]['GSO']

# Convert to sets for comparison
set1 = set(mdh_list_1)
set2 = set(mdh_list_2)

# Find intersections and differences
common = set1 & set2
only_in_1 = set1 - set2
only_in_2 = set2 - set1

# Show results side by side in a DataFrame
import pandas as pd

df_compare = pd.DataFrame({
    "Only in (2058,rcp45hotter)": sorted(list(only_in_1)),
    "Only in (2058,rcp85hotter)": sorted(list(only_in_2)),
})

print("Common MDH (present in both):")
print(sorted(list(common)))
print("\nDifferences side by side:")
print(df_compare)

In [4]:
import pandas as pd
import matplotlib.pyplot as plt


def plot_mdh_calendar_grid(list_of_mdh, city, key1, key2, dummy_year=2000):
    mdh_list_1 = list_of_mdh[key1][city]
    mdh_list_2 = list_of_mdh[key2][city]

    set1, set2 = set(mdh_list_1), set(mdh_list_2)
    both  = sorted(set1 & set2, key=lambda t: (t[0], t[1], t[2]))
    only1 = sorted(set1 - set2, key=lambda t: (t[0], t[1], t[2]))
    only2 = sorted(set2 - set1, key=lambda t: (t[0], t[1], t[2]))

    def to_dt(triple):
        m, d, h = triple
        return pd.Timestamp(dummy_year, m, d, h)

    # Prepare x (datetime) and y (hour)
    x1 = [to_dt(t) for t in only1]
    y1 = [ts.hour for ts in x1]

    xb = [to_dt(t) for t in both]
    yb = [ts.hour for ts in xb]

    x2 = [to_dt(t) for t in only2]
    y2 = [ts.hour for ts in x2]

    plt.figure(figsize=(11, 5))
    plt.scatter(x1, y1, label=f"Only {key1}", s=14)
    plt.scatter(xb, yb, label="Both",        s=14)
    plt.scatter(x2, y2, label=f"Only {key2}", s=14)

    plt.gca().set_ylim(-0.5, 23.5)
    plt.gca().invert_yaxis()  # midnight at bottom (optional)
    plt.yticks(range(0, 24, 2))

    # Format x-axis as month-day, in correct chronological order
    plt.gca().set_xticks(sorted(set(x1 + xb + x2)))
    plt.gca().set_xticklabels(
        [dt.strftime("%m-%d") for dt in sorted(set(x1 + xb + x2))],
        rotation=45, ha="right"
    )

    plt.xlabel("Date (MM-DD)")
    plt.ylabel("Hour of day")
    plt.title(f"MDH Calendar Grid — {city}: {key1} vs {key2}")
    plt.legend()
    plt.tight_layout()
    plt.show()

# Example call:
plot_mdh_calendar_grid(list_of_mdh, 'GSO', ('2018','historical'), ('2058','rcp85hotter'))

# Conversion script

In [5]:
# --- Define parameters for resistance model ---
T0 = 20 # default temperature for resistance values in SMART-DS
alpha_r = 0.00403 # temperature coefficient default for power line 
f_amp_temp = 0.012 # ampacity change per degree celsuis (%/C)
near_worst_stat = 'p99' # options: avg_daily_max_august | p98 | p99 | p995 | p999 (assumption for ampacity near-worst weather condition baseline. p99 is typically used by utilities and is the CIGRE guideline)
regional_peak_start_month = 6; regional_peak_end_month = 9 # inputs to get_regional_peak_times() function, which finds the regional peak timestep during these months

start_time = time.time()

print(f"--- Solution mode: {solution_mode} Demand mode:{demand_mode} Line rating mode:{line_rating_mode} --- \n")

# Exit program if solution_mode is different than snapshot mode
if solution_mode != "snapshot":
    sys.stderr.write(f"ERROR: solution_mode must be 'snapshot', got {solution_mode!r}.\n")
    sys.exit(1)

for TGW_weather_year, TGW_scenarios in TGW_years_scenarios.items():
    for TGW_scenario in TGW_scenarios:
        print(f"--- TGW_scenario:{TGW_scenario} TGW_weather_year:{TGW_weather_year} --- \n")
        for city, regions in CITY_REGIONS_TO_RUN.items():
            
            # Load TGW weather data for TGW location (city)
            TGW_location = {"GSO": "Greensboro","AUS": "Austin","SFO": "SanFrancisco"}.get(city, city)
            TGW_weather_df_save_path = f"{input_data_prediction_path}/{TGW_location}/{TGW_scenario}/"
            TGW_weather_df = joblib.load(os.path.join(TGW_weather_df_save_path,f"TGW_weather_{TGW_weather_year}.joblib"))
            
            # load near-worst historical temperature for TGW location (city) to set default ambient temp of power lines ampacity
            TGW_stats_dir = f"main_folder/TGW/TGW_Distribution_for_Aviad/TGW_weather_statistics"
            loaded_temp_stats = joblib.load(os.path.join(TGW_stats_dir, f"temperature_stats.joblib"))
            Ta_near_worst = loaded_temp_stats[TGW_location].loc[(loaded_temp_stats[TGW_location]['scenario'] == 'historical') & (loaded_temp_stats[TGW_location]['year'] == int(smart_ds_year)), near_worst_stat].values[0]
            print(f"Near-worst ({near_worst_stat}) historical local temperature found for TGW_scenario historical TGW_weather_year {smart_ds_year} {city} is {Ta_near_worst} (used as base temp for line derating)\n")

            ## create list of mdh for top % hours
            df_city = regional_demand_weather_ampacity_all_cities_sorted[(TGW_weather_year, TGW_scenario)][city]   # load regional demand data to later create list of mdh for top % hours  
            list_of_mdh = df_ops.get_top_n_mdh(df_city, top_n_hours, start_month_mdh, end_month_mdh)

            for region in regions:
                print(f"--- city: {city}, region: {region} ---\n")

                region_start_time = time.time()
                
                # Define path to opendss region folder (assuming snapshot mode)
                region_path = f"main_folder/SMART-DS/v1.0/{smart_ds_year}/{city}/{region}/scenarios/base_timeseries/opendss_no_loadshapes"

                # --- Load demand data (regional data at the buildings level) ---
                ### Load measured buildings' demand timeseries dataframe, organized by feeders (single region) | colums: total_site_electricity_kw	pf	cooling_sum_kw	heating_kw	non_cool_n_heat_kw
                input_data_region_dir = f'{smart_ds_load_path}/{city}/{region}/buildings'
                measured_buildings_cool_heat_dict = joblib.load(os.path.join(input_data_region_dir, "measured_buildings_cool_heat_dict.joblib")) # Save the file
                ### Load predicted buildings' total demand panda timeseries, organized by feeders (single region) 
                predictions_path = f"main_folder/load_prediction/results/data/prediction/output/{config['smart_ds_years'][0]}/months_{config['start_month']}_{config['end_month']}/cooling_n_heating/{config['X_columns_set']}/{config['aggregation_level']}/" 
                predictions_dir = os.path.join(predictions_path, f"{TGW_scenario}/predictions/{city}/{region}/")
                building_predicted_total_dict = joblib.load(os.path.join(predictions_dir, f"{demand_mode}_TGW_{TGW_weather_year}_buildings_dict.joblib"))

                print(f"Loaded measured and predicted buildings' demand timeseries\n")

                # --- Pre-Process data (Convert ResStock building kw timeseries to smart-ds kw and kvar loadshapes and max values) ---

                ### Create predicted kvar from predicted kw and measured pf - create predicted reactive load profiles (kvar) using measured power factor time series and predicted kw | each resulting df has columns: date_time kw kvar
                for outer_key, inner_dict in building_predicted_total_dict.items():   
                    for building_name, kw_series in inner_dict.items():
                        # Convert active power series to DataFrame
                        df = kw_series.to_frame(name='kw')

                        # Get the matching power factor series
                        pf_series = measured_buildings_cool_heat_dict[outer_key][building_name]['pf']

                        # Align indices if needed
                        pf_series = pf_series.loc[df.index]

                        # Calculate reactive power
                        angle_rad = np.arccos(pf_series.clip(lower=0.01, upper=1.0))  # prevent domain error
                        df['kvar'] = df['kw'] * np.tan(angle_rad)

                        # Store the result
                        building_predicted_total_dict[outer_key][building_name] = df

                print("Created predicted kvar from predicted kw and measured pf \n")

                ### Create measured kw and kvar max from measured kw and pf - Use measured kw and measured power factor time series to create measured kw and kvar max
                # Initialize dictionary to store max values
                building_measured_max_dict = {} 
                for outer_key, inner_dict in measured_buildings_cool_heat_dict.items():
                    building_measured_max_dict[outer_key] = {}

                    for building_name, df in inner_dict.items():

                        # compute kvar timeseries
                        # Get the matching power factor series
                        pf_series = measured_buildings_cool_heat_dict[outer_key][building_name]['pf']
                        pf_series = pf_series.loc[df.index] # Align indices if needed

                        # Calculate reactive power
                        angle_rad = np.arccos(pf_series.clip(lower=0.01, upper=1.0))  # prevent domain error
                        df['total_site_electricity_kvar'] = df['total_site_electricity_kw'] * np.tan(angle_rad)

                        # Compute max values
                        kw_max = df['total_site_electricity_kw'].max()
                        kvar_max = df['total_site_electricity_kvar'].max()

                        # Store max values
                        building_measured_max_dict[outer_key][building_name] = {
                            'kw_max': kw_max,
                            'kvar_max': kvar_max
                        }

                print("Created measured kw and kvar max from measured kw and pf \n")
                
                # Free unused memory
                del measured_buildings_cool_heat_dict
                gc.collect()
                
                ## Get lists of months, days and hours from a sample dataframe in building_predicted_total_dict (used later to mask selected mdh)
                first_outer_key, first_inner_dict = next(iter(building_predicted_total_dict.items())) # get the first outer_key and its inner dict
                first_building_name, first_df = next(iter(first_inner_dict.items())) # get the first building_name and df
                months = df.index.month; days   = df.index.day; hours  = df.index.hour
                
                # Initialize dictionary to store max values
                building_predicted_max_dict = {}
                                
                # find regional kw and kvar peak timestep (timestamp) using building_predicted_total_dict
                regional_peak_times = df_ops.get_regional_peak_times(building_predicted_total_dict, regional_peak_start_month, regional_peak_end_month)
                print(f"Extracted regional peak time: {regional_peak_times}\n")

                for outer_key, inner_dict in building_predicted_total_dict.items():
                    building_predicted_max_dict[outer_key] = {}

                    for building_name, df in inner_dict.items():
                        # Compute max values
                        kw_max = df['kw'].max()
                        kvar_max = df['kvar'].max()

                        # Compute kw kvar values at regional peak timestep
                        kw_at_region_peak = df['kw'].loc[regional_peak_times['kw_regional_peak_time']]
                        kvar_at_region_peak = df['kvar'].loc[regional_peak_times['kvar_regional_peak_time']]

                        # Store max values
                        building_predicted_max_dict[outer_key][building_name] = {
                            'kw_max': kw_max,
                            'kvar_max': kvar_max,
                            'kw_at_region_peak': kw_at_region_peak,
                            'kvar_at_region_peak': kvar_at_region_peak,
                        }

                print("Converted predicted kw and kvar timeseries to load shapes (0-1) values, max values and values at the regional peak\n")
                

                dss.Command(f'Redirect "{region_path}/Master.dss"')
                print(f"Redirected dss engine to {region_path}/Master.dss")
                
                # Organize all building_type entries under each feeder (for Load.dss process)
                feeder_dict = defaultdict(dict)
                for key, building_dict in building_predicted_total_dict.items():
                    smart_ds_year, city, region, feeder, building_type = key
                    feeder_key = (smart_ds_year, city, region, feeder)
                    feeder_dict[feeder_key][building_type] = building_dict
                                             

                # Create a list of paths to original folders with linecodes.dss / Transformers.dss / Loads.dss (that's why we use max depth 3)
                folders_with_linecodes = file_ops.find_folders_with_file(region_path, "LineCodes.dss", max_depth=3)
                folders_with_transformers = file_ops.find_folders_with_file(region_path, "Transformers.dss", max_depth=3)
                # Get list of original feeder paths (folders with Loads.dss)
                feeder_folders = file_ops.find_folders_with_file(region_path, "Loads.dss")

                # --- Iterate over all selected month-day-hour (mdh) --- 
                for row_i in range(start_row_idx, end_row_idx):
                    mdh = list_of_mdh[row_i]; m,d,h = mdh
                    print(f"Creating files for month:{m} hour:{d} day:{h}\n")

                    mdh_mask = (months == m) & (days == d) & (hours == h) # mask to select kw value for each building at the selected mdh

                    ## Extract temperature selected time 
                    # Filter TGW_weather_df rows by matching month, day, and hour
                    matched = TGW_weather_df[(TGW_weather_df['date_time'].dt.month == m) & (TGW_weather_df['date_time'].dt.day == d) & (TGW_weather_df['date_time'].dt.hour == h)]
                    # Extract temperature at m, d, h
                    Ta = int(matched[['Dry Bulb Temperature [°C]']].iloc[0, 0])
                    print(f"Found Temperature for TGW_scenario {TGW_scenario} TGW_weather_year {TGW_weather_year} {city} {region}: {Ta} (used for derating and resistance)\n")

                    # Modify lines' thermal capacity and resistance in LineCodes files [currently uses an approximation for ampacity change]
                    # Calculate resistance and ampacity temperature-based factors
                    Rmatrix_factor = 1 + (alpha_r * (Ta - T0)) # Define Rmatrix multiplier based on T-R relationship
                    amp_factor = (1 - ((Ta - Ta_near_worst) * f_amp_temp) )

                    ## ****** modify functions to change name convention to finish with _m_d_h.dss ****** ##
                    for folder_path in folders_with_linecodes:
                        # opendss_ops.modify_LineCodes(folder_path, Rmatrix_factor, Ta)
                        opendss_ops.modify_LineCodes(folder_path, Rmatrix_factor, amp_factor, line_rating_mode, TGW_scenario, TGW_weather_year,m,d,h)
                    # Modify transformers' thermal capacity (KVA rating values) in Transformers files  [using IEEEc57.91 Table 3 approximation] 
                    for folder_path in folders_with_transformers:
                        opendss_ops.modify_tranformers(folder_path, Ta, transformers_rating_mode, TGW_scenario, TGW_weather_year,m,d,h)
                    print(f"Saved modified line codes and transformers")


                    ### Create load.dss with load at selected mdh 
                    phase_load_max_dict = {}  # Initialize phase load dictionary 
                    # Iterate over each feeder once (rather than once per building type, like other dictionaries in this process)
                    for feeder_key, building_types_dict in feeder_dict.items():
                        smart_ds_year, city, region, feeder = feeder_key
                        phase_load_max_dict[feeder_key] = {}  # Initialize phase load internal dictionary, e.g., feeder_key = ('2018', 'GSO', 'rural', 'rhs2_1247--rdt1262')
                        # use feeder_key and feeder_folders to get path to loads.dss
                        # Use regex to find the exact feeder folder
                        feeder_pattern = re.compile(rf'(^|/){re.escape(feeder)}(/|$)')
                        matching_feeders = [f for f in feeder_folders if feeder_pattern.search(f)]
                        if not matching_feeders:
                            print(f"[WARNING] Feeder folder not found for feeder: {feeder}")
                            continue
                        feeder_folder = matching_feeders[0]
                        path_to_loads_dss = os.path.join(feeder_folder,"Loads.dss") # path to original Loads.dss, e.g., main_folder/SMART-DS/v1.0/2018/GSO/rural/scenarios/base_timeseries/opendss/rhs2_1247/rhs2_1247--rdt1262/Loads.dss

                        # create scenario-based path (Feeder_folder/predicted_loads/TGW/climate_scnario/weather_year/)
                        output_dir = os.path.join(feeder_folder, "predicted_loads", "TGW", TGW_scenario, TGW_weather_year)
                        os.makedirs(output_dir, exist_ok=True)

                        # Create copy of loads.dss in scenario-based path
                        original_loads_path = path_to_loads_dss # Path to original loads file

                        ## ****** change name convention to finish with _m_d_h.dss****** ##
                        new_loads_path = output_dir + f"/Loads_{m}_{d}_{h}.dss"    # Path to new loads file

                        shutil.copyfile(original_loads_path, new_loads_path) # Duplicate loads file

                        with open(original_loads_path, "r") as infile, open(new_loads_path, "w") as outfile:
                            for line in infile:     ## Loop over all rows in loads.dss (all phase loads)
                                if line.startswith("New Load."):
                                    # --- Extract data from the Loads.dss line ---
                                    # Extract resstock building name (in the format of loadShape names, e.g., res_kw_626_pu)
                                    loadshape_pattern = re.search(r'yearly=(\w+)_(kw|kvar)_(\d+)_pu', line) # res_kw_452_pu returns: loadshape_pattern.group(1) = res | loadshape_pattern.group(2) = kw | loadshape_pattern.group(3) = 452 
                                    # Extract load phase name, e.g., load_p1rlv5636_2
                                    phase_load_name_pattern = re.search(r'Load\.(\S+)\s', line)
                                    # Make sure patterns were found
                                    if not loadshape_pattern or not phase_load_name_pattern:
                                        raise ValueError("Failed to extract required fields from Loads.dss line")
                                    # Use patterns
                                    building_loadshape_name = f"{loadshape_pattern.group(1)}_{loadshape_pattern.group(2)}_{loadshape_pattern.group(3)}_pu_{feeder}" # e.g., res_kw_452_pu
                                    phase_load_name = phase_load_name_pattern.group(1)
                                    # Extract measured kw kvar load phase values 
                                    pattern = r'kW=([+-]?[0-9]*\.?[0-9]+(?:[eE][+-]?[0-9]+)?)\s+kvar=([+-]?[0-9]*\.?[0-9]+(?:[eE][+-]?[0-9]+)?)'  # Regex pattern to find kW and kvar values
                                    # Search for pattern
                                    match = re.search(pattern, line)
                                    if match:
                                        measured_load_phase_kw_max = float(match.group(1))
                                        measured_load_phase_kvar_max = float(match.group(2))

                                    # Initialize inner dictionary using phase load name
                                    phase_load_max_dict[feeder_key][phase_load_name] = {}

                                    # Get building type and name and outer key
                                    building_name = f"{loadshape_pattern.group(1)}_{loadshape_pattern.group(3)}" # , e.g., res_626
                                    building_type = building_name.split('_')[0] # e.g., res
                                    outer_key = (feeder_key[0],feeder_key[1],feeder_key[2],feeder_key[3],building_type)

                                    ## ******  use building_name to get kw kvar at selected mdh  ****** ##
                                    # Add data to phase load dictionary: phase load name | resstock name |  measured load phase kw max | measured load phase kvar max
                                    phase_load_max_dict[feeder_key][phase_load_name]['phase_load_name'] = phase_load_name
                                    phase_load_max_dict[feeder_key][phase_load_name]['building_name'] = building_name
                                    phase_load_max_dict[feeder_key][phase_load_name]['load_shape_name'] = building_loadshape_name
                                    phase_load_max_dict[feeder_key][phase_load_name]['measured_kw'] = measured_load_phase_kw_max
                                    phase_load_max_dict[feeder_key][phase_load_name]['measured_kvar'] = measured_load_phase_kvar_max

                                    # load measured and predicted resstock kw and kvar max
                                    measured_ressstock_kw_max = building_measured_max_dict[outer_key][building_name]['kw_max']
                                    measured_ressstock_kvar_max = building_measured_max_dict[outer_key][building_name]['kvar_max']


                                    # Load predicted resstock kw and kvar max and value at regional peak timestep
                                    predicted_ressstock_kw_at_region_peak = building_predicted_max_dict[outer_key][building_name]['kw_at_region_peak'] 
                                    predicted_ressstock_kvar_at_region_peak = building_predicted_max_dict[outer_key][building_name]['kvar_at_region_peak']
                                    predicted_ressstock_kw_max = building_predicted_max_dict[outer_key][building_name]['kw_max'] 
                                    predicted_ressstock_kvar_max = building_predicted_max_dict[outer_key][building_name]['kvar_max'] 


                                    # Compute kw kvar values at selected mdh
                                    building_df = building_predicted_total_dict[outer_key][building_name]
                                    mdh_mask = (months == m) & (days == d) & (hours == h)
                                    kw_vals = building_df.loc[mdh_mask, 'kw'] # get all kw with mdh match to mask
                                    kvar_vals = building_df.loc[mdh_mask, 'kvar'] # get all kvar  with mdh match to mask
                                    predicted_ressstock_kw_at_mdh = kw_vals.iloc[0] # kw value at first mdh match 
                                    predicted_ressstock_kvar_at_mdh = kvar_vals.iloc[0] # kvar value at first mdh match 
                                    
                                    predicted_ressstock_kw_at_mdh = kw_vals.iloc[0] # kw value at first mdh match 
                                    predicted_ressstock_kvar_at_mdh = kvar_vals.iloc[0] # kvar value at first mdh match 

                                    # Calculate predicted load phase kw kvar values 
                                    if measured_ressstock_kw_max == 0 or measured_ressstock_kvar_max == 0:
                                        raise ValueError(f"[ERROR] Zero measured max for {building_name} in {feeder_key}")
                                    predicted_load_phase_kw_max = predicted_ressstock_kw_max * (measured_load_phase_kw_max / measured_ressstock_kw_max)
                                    predicted_load_phase_kvar_max = predicted_ressstock_kvar_max * (measured_load_phase_kvar_max / measured_ressstock_kvar_max)
                                    predicted_load_phase_kw_at_region_peak = predicted_ressstock_kw_at_region_peak * (measured_load_phase_kw_max / measured_ressstock_kw_max)
                                    predicted_load_phase_kvar_at_region_peak = predicted_ressstock_kvar_at_region_peak * (measured_load_phase_kvar_max / measured_ressstock_kvar_max)
                                    predicted_load_phase_kw_at_mdh = predicted_ressstock_kw_at_mdh * (measured_load_phase_kw_max / measured_ressstock_kw_max)
                                    predicted_load_phase_kvar_at_mdh = predicted_ressstock_kvar_at_mdh * (measured_load_phase_kvar_max / measured_ressstock_kvar_max)

                                    # Add predicted kw kvar values to dictioanry
                                    # data in dictionary will be: phase load name | resstock name |  measured load phase kw max | measured load phase kvar max | predicted load phase kw max | predicted load phase kvar max | predicted load phase kw regional peak | predicted load phase kvar regional peak
                                    phase_load_max_dict[feeder_key][phase_load_name]['predicted_kw_max'] = predicted_load_phase_kw_max
                                    phase_load_max_dict[feeder_key][phase_load_name]['predicted_kvar_max'] = predicted_load_phase_kvar_max
                                    phase_load_max_dict[feeder_key][phase_load_name]['predicted_kw_at_region_peak'] = predicted_load_phase_kw_at_region_peak
                                    phase_load_max_dict[feeder_key][phase_load_name]['predicted_kvar_at_region_peak'] = predicted_load_phase_kvar_at_region_peak
                                    phase_load_max_dict[feeder_key][phase_load_name]['predicted_kw_at_mdh'] = predicted_load_phase_kw_at_mdh
                                    phase_load_max_dict[feeder_key][phase_load_name]['predicted_kvar_at_mdh'] = predicted_load_phase_kvar_at_mdh

                                    predicted_load_phase_kw = predicted_load_phase_kw_at_mdh
                                    predicted_load_phase_kvar = predicted_load_phase_kvar_at_mdh

                                    # Modify loads.dss line with predicted kw kvar values and new resstock load shape names 'res_#_feeder' (add feeder name to existing yearly='')
                                    # Update kW and kvar values in the line
                                    line = re.sub(r"kW=([0-9\.]+)", lambda m: f"kW={predicted_load_phase_kw}", line)
                                    line = re.sub(r"kvar=([0-9\.]+)", lambda m: f"kvar={predicted_load_phase_kvar}", line)
                                    line = re.sub(r'(yearly=[^\s\n]+)', r'\1_' + feeder, line)

                                # Modify loads.dss in scenario-based path with the new line
                                outfile.write(line)
                    print(f"Saved modified Load.dss files\n")

                    ## Create a duplicate master file with paths to new load.dss and loadshapes.dss
                    original_master_file = region_path + "/Master.dss" # Path to original master file
                    new_master_dir = os.path.join(region_path, "predicted_master_files", TGW_scenario, TGW_weather_year)
                    os.makedirs(new_master_dir, exist_ok=True)                 # Create directory if it does not exist
                    new_master_file = new_master_dir + f"/Master_{TGW_scenario}_{TGW_weather_year}_{m}_{d}_{h}.dss"    # Path to new master file
                    shutil.copyfile(original_master_file, new_master_file) # Duplicate master file
                    opendss_ops.modify_master_file(new_master_file, solution_mode, demand_mode, line_rating_mode, transformers_rating_mode, TGW_scenario, TGW_weather_year,m,d,h)

                    print(f"A modified master file was created in {new_master_file}\n")
  
                # Free unused memory
                del phase_load_max_dict 
                gc.collect() 
                
                # Free unused memory after every region run
                del building_predicted_total_dict
                del building_predicted_max_dict
                gc.collect()
                
                region_end_time = time.time(); print(f"---Runtime for {city} {region}: {(region_end_time - region_start_time) / 60:.2f} minutes---\n")
                
end_time = time.time(); print(f"--- Total Runtime: {(end_time - start_time) / 60:.2f} minutes ---")